In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

google_df = pd.read_excel('GooglePlaystore.xlsx')
google_df = google_df[google_df['Reviews'] != '3.0M']
google_df = google_df.replace("Varies with device", pd.NA)
google_df = google_df.dropna()

def clean_android_ver(val):
    if '-' in val:
        val = val.split('-')[0].strip()
    val = val.replace('and up', '').strip()
    parts = val.split('.')
    return '.'.join(parts[:2])
google_df['Android Ver'] = google_df['Android Ver'].apply(clean_android_ver)

google_df['Installs'] = google_df['Installs'].str.replace('[+,]', '', regex=True)
google_df = google_df[google_df['Installs'].str.isnumeric()]
google_df['Installs'] = google_df['Installs'].astype(int)

google_df['Reviews'] = google_df['Reviews'].astype(int)
google_df['Rating'] = pd.to_numeric(google_df['Rating'], errors='coerce')
condition = (google_df['Reviews'] < 100) & (google_df['Installs'] < 50000)
google_df = google_df[~((google_df['Rating'].isna()) & condition)]
google_df['Rating'] = google_df.groupby('Category')['Rating'].transform(lambda x: x.fillna(round(x.mean(), 2)))

def convert_size(val):
    val = val.upper().replace(',', '').strip()
    if 'K' in val:
        return int(float(val.replace('K','')) * 1000)
    elif 'M' in val:
        return int(float(val.replace('M','')) * 1000000)
    else:
        try:
            return int(float(val))
        except:
            return pd.NA
google_df['Size'] = google_df['Size'].apply(convert_size)
google_df = google_df.dropna(subset=['Size'])

rating_stats = google_df.groupby('Category')['Rating'].describe()
print(rating_stats)

free_apps = google_df[google_df['Type'] == 'Free']
def top3_by(metric):
    return (free_apps.sort_values([metric], ascending=False)
                   .groupby('Category')
                   .head(3)
                   [['Category', 'App', metric]])
print(top3_by('Rating').head(10))
print(top3_by('Installs').head(10))
print(top3_by('Reviews').head(10))

paid_apps = google_df[google_df['Type'] == 'Paid']
paid_apps['Price'] = paid_apps['Price'].astype(str).str.replace('$', '').astype(float)
print(paid_apps['Price'].mean(), paid_apps['Price'].max(), paid_apps['Price'].min())

google_df['Genres'] = google_df['Genres'].str.split(';')
genre_df = google_df.explode('Genres')
genre_counts = genre_df['Genres'].value_counts()
plt.figure(figsize=(8,8))
genre_counts.head(10).plot(kind='pie', autopct='%1.1f%%')
plt.ylabel('')
plt.title('Top 10 Genres Distribution')
plt.show()

subset = google_df[google_df['Category'].isin(['BUSINESS', 'EDUCATION'])]
plt.figure(figsize=(6,6))
sns.boxplot(x='Category', y='Rating', data=subset)
plt.title('Ratings: Business vs Education')
plt.show()
